<a href="https://colab.research.google.com/github/blongho/ai-for-developers/blob/master/chatgpt_for_developers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install openai

## Setup

In [ ]:
#!pip install openai
from openai import OpenAI
import os

## Get the Licence key

### Getting your keys from local .env file

In [ ]:
"""
!pip install dotenv

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')
"""

"\n!pip install dotenv \n\nfrom dotenv import load_dotenv, find_dotenv\n_ = load_dotenv(find_dotenv()) # read local .env file\n\nopenai.api_key  = os.getenv('OPENAI_API_KEY')\n"

### Getting your keys stored in your colab workspace.
At the left pannel, click on the key symbol then "Add new secret". Enter "OPEN_API_KEY" in the Name field and your api key in the Value section

In [ ]:
from google.colab import userdata

In [ ]:
client = OpenAI(
  api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
def get_completion(prompt, model="gpt-4o", temperature=0):
  response = client.chat.completions.create(
    model=model,
    messages = [{"role": "user", "content": prompt}],
    temperature=temperature, # this is the degree of randomness of the model's output
    store=True,
  )
  return response.choices[0].message.content

## Principle 1: Write clear and specific Instructions

### 1.1: Use delimiters to specify when the texts end

In [ ]:
  text=f"""
  You should express what you think a model to do by \
  providing instructions that are as clear and \
  specific as you can possibly make them. \
  This will guide the model towards the desired output, \
  and reduce the chances of receiving irrelevant \
  or incorrect responses. Don't confuse writing a \
  clear prompt with writing a short prompt. \
  In many cases, longer prompts provide more clarity \
  and context for the model, which can lead to \
  more detailed and relevant outputs.
  """
  prompt=f"""
  Summarize the text delimited by triple bacttics \
  into a single sentence.
  ```{text}```
  """
  response=get_completion(prompt)
  print(response)

To guide a model effectively and reduce irrelevant responses, provide clear and specific instructions, as longer prompts often offer better clarity and context.


### 1.2: Ask for structured text

In [ ]:
prompt = f"""
Generate a list of three made-up book titles along \
with their authors and genres.
Provide them in JSON format with the following keys:
book_id, title, author, genre.
"""
json_response = get_completion(prompt)

In [ ]:
print(json_response)

```json
[
    {
        "book_id": 1,
        "title": "Whispers of the Forgotten Forest",
        "author": "Elara Moonstone",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "The Quantum Enigma",
        "author": "Dr. Orion Voss",
        "genre": "Science Fiction"
    },
    {
        "book_id": 3,
        "title": "Echoes of a Silent City",
        "author": "Mira Solstice",
        "genre": "Mystery"
    }
]
```


### 1.3: Check whether conditions are satisfied. Check assumptions required to do the task

In [ ]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some water boiling. While that's happening, \
grab a cup and put a tea bag in it. Once the water is hot enough, just pour it over the tea bag. \
Let it sit for a bit so the tea can steep. After a few minutes, take out the tea bag. If you \
like, you can add some sugar or milk to taste. And that's it! You've got yourself a delicious \
cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.  
Step 2 - Grab a cup and put a tea bag in it.  
Step 3 - Once the water is hot enough, pour it over the tea bag.  
Step 4 - Let it sit for a bit so the tea can steep.  
Step 5 - After a few minutes, take out the tea bag.  
Step 6 - If you like, add some sugar or milk to taste.  
Step 7 - Enjoy your delicious cup of tea.


In [ ]:
text_2 = f"""
The sun is shining brightly today, and the birds are singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


### 1.4: Few Shot Prompting

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest valley flows from a modest spring; the \
grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Consider the mighty oak that stands tall through countless storms; its strength is not in resisting the wind, but in bending with it. The diamond, too, is born from immense pressure, emerging more brilliant than before. Resilience is the art of enduring and adapting, growing stronger with each challenge faced.


## Principle 2: Give the model time to think

### 2.1: Specify the steps required to complete the task

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Jack and Jill, siblings from a charming village, embarked on a quest to fetch water from a hilltop well, but misfortune struck when Jack tripped and fell, with Jill following, yet they returned home slightly battered but undeterred in their adventurous spirits.

Jack et Jill, frères et sœurs d'un charmant village, se sont lancés dans une quête pour aller chercher de l'eau à un puits au sommet d'une colline, mais la malchance a frappé lorsque Jack a trébuché et est tombé, suivi de Jill, mais ils sont rentrés chez eux légèrement meurtris mais pas découragés dans leur esprit d'aventure.

Jack, Jill

{
  "french_summary": "Jack et Jill, frères et sœurs d'un charmant village, se sont lancés dans une quête pour aller chercher de l'eau à un puits au sommet d'une colline, mais la malchance a frappé lorsque Jack a trébuché et est tombé, suivi de Jill, mais ils sont rentrés chez eux légèrement meurtris mais pas découragés dans leur esprit d'aventure.",
  "num_names": 2
}

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop \
well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled \
down the hill, with Jill following suit. Though slightly battered, the pair returned home to \
comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they \
continued exploring with delight.
"""
# example 1
prompt_2 = f"""
Perform the following actions:
1 - Summarize the following text delimited by triple backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>
Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_2)
print("Completion for prompt 2:")
print(response)

Completion for prompt 2:
Summary: In a charming village, siblings Jack and Jill embarked on a quest to fetch water from a hilltop well, but after Jack tripped and fell, followed by Jill, they returned home slightly battered yet undeterred in their adventurous spirits.

Translation: Dans un charmant village, les frères et sœurs Jack et Jill se sont lancés dans une quête pour aller chercher de l'eau à un puits au sommet d'une colline, mais après que Jack ait trébuché et soit tombé, suivi par Jill, ils sont rentrés chez eux légèrement meurtris mais toujours déterminés dans leur esprit d'aventure.

Names: Jack, Jill

Output JSON: 
```json
{
  "french_summary": "Dans un charmant village, les frères et sœurs Jack et Jill se sont lancés dans une quête pour aller chercher de l'eau à un puits au sommet d'une colline, mais après que Jack ait trébuché et soit tombé, suivi par Jill, ils sont rentrés chez eux légèrement meurtris mais toujours déterminés dans leur esprit d'aventure.",
  "num_names":

### 2.2: Instruct the model to work out its own solution before rusing to a conclusion

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is mostly correct, but there is a small mistake in the maintenance cost calculation. The maintenance cost should be $100,000 plus $10 per square foot, not $100 per square foot. Let's correct that:

1. Land cost: \(100x\)
2. Solar panel cost: \(250x\)
3. Maintenance cost: \(100,000 + 10x\)

The total cost for the first year of operations as a function of the number of square feet \(x\) should be:

\[100x + 250x + 100,000 + 10x = 360x + 100,000\]

So, the correct total cost function is \(360x + 100,000\).


In [ ]:
prompt = f"""
Your task is to determine if the student's solution is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

```
Let x be the size of the installation in square feet.

1. Land cost: The cost of the land is $100 per square foot, so for x square feet, the cost is:
   Land cost = 100x

2. Solar panel cost: The cost of the solar panels is $250 per square foot, so for x square feet, the cost is:
   Solar panel cost = 250x

3. Maintenance cost: The maintenance cost consists of a flat fee of $100,000 per year plus an additional $10 per square foot. Therefore, for x square feet, the maintenance cost is:
   Maintenance cost = 100,000 + 10x

The total cost for the first year of operations is the sum of the land cost, solar panel cost, and maintenance cost:
   Total cost = Land cost + Solar panel cost + Maintenance cost
              = 100x + 250x + (100,000 + 10x)
              = 100x + 250x + 100,000 + 10x
              = 360x + 100,000
```
Is the student's solution the same as actual solution just calculated:
```
No
```
Student grade:
```
Incorrect
```


## Model Limitations

### Hallucination

Model can make statements that sound plausible but are not true.

### Reduce hallucinations by
- First find relevant information then answer the question based on the relevant information

In [ ]:
prompt = """
Tell me about Longho Lodge Bonduma - Buea.
"""
response = get_completion(prompt)
print(response)

Longho Lodge is a guesthouse located in Bonduma, a neighborhood in Buea, Cameroon. Buea is a city situated at the foot of Mount Cameroon, the highest mountain in West Africa, and is known for its cool climate and historical significance as the former capital of German Kamerun. 

Longho Lodge offers accommodations that cater to travelers looking for a comfortable and convenient place to stay while exploring the region. The lodge typically provides basic amenities such as clean rooms, private bathrooms, and possibly some common areas for guests to relax. The location in Bonduma makes it accessible to various local attractions, including hiking trails on Mount Cameroon, the University of Buea, and cultural sites in the city.

Guests at Longho Lodge can enjoy the scenic views of the surrounding landscape and the opportunity to experience the local culture and hospitality. The lodge may also offer services such as meals, guided tours, or transportation arrangements to enhance the stay of it

## Iterative Prompt Development

In [ ]:
# Generating a market product description from a product fact sheet
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Elevate your workspace with our exquisite mid-century inspired office chair, a perfect blend of style and functionality. Part of a stunning collection of office furniture, this chair is designed to complement any home or business setting with its versatile options and elegant design.

Crafted in Italy, this chair offers a range of customizable features to suit your personal taste and needs. Choose from a variety of shell colors and base finishes, including stainless steel, matte black, gloss white, or chrome, to create a look that is uniquely yours. The chair is available in two upholstery options: the SWC-100 with a plastic back and front upholstery, or the SWC-110 with full upholstery, available in 10 fabric and 6 luxurious leather options.

Designed for comfort and durability, the chair features a 5-wheel plastic coated aluminum base and a pneumatic adjuster for easy height adjustments. The seat is made with high-quality HD36 foam, available in two densities for personalized comfort

### Issue 1: Text too long. Set a limit

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Use at most 50 words

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Elevate your workspace with our mid-century inspired office chair, available in various shell colors and base finishes. Choose from plastic or full upholstery in 10 fabrics and 6 leathers. Features include a 5-wheel aluminum base, pneumatic height adjustment, and optional armrests. Perfect for home or business use. Made in Italy.


In [ ]:
len(response.split(" "))

50

### Limit the sentences

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Use at most 3 sentences

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Elevate your workspace with our mid-century inspired office chair, designed for both home and business settings. Choose from a variety of shell colors, base finishes, and upholstery options, including 10 fabrics and 6 leathers, to perfectly match your style. With features like a 5-wheel plastic coated aluminum base, pneumatic height adjustment, and optional armrests, this Italian-crafted chair combines elegance with functionality.


### Specify the target audience

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

This mid-century inspired office chair features a durable cast aluminum shell with a modified nylon PA6/PA66 coating. It offers a 5-wheel plastic-coated aluminum base and pneumatic height adjustment. Choose from various upholstery options and base finishes, with optional armrests and seat foam densities. Made in Italy.


### Extract relevant and extra information

In [ ]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

After the description, include a table that gives the
product's dimensions. The table should have two columns.
In the first column include the name of the dimension.
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)

In [ ]:
from IPython.display import display, HTML

In [ ]:
display(HTML(response))

Dimension,Measurement (inches)
Width,20.87”
Depth,20.08”
Height,31.50”
Seat Height,17.32”
Seat Depth,16.14”


## Summarizing

### Product review

In [ ]:
# text

In [ ]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \
super cute, and its face has a friendly look. It's \
a bit small for what I paid though. I think there \
might be other options that are bigger for the \
same price. It arrived a day earlier than expected, \
so I got to play with it myself before I gave it \
to her.
"""

In [ ]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site.

Summarize the review below, delimited by triple
backticks, in at most 30 words.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The panda plush toy is soft, cute, and loved by the daughter, but considered small for the price. It arrived earlier than expected.


### Summarize with a focus on shipping and delivery

In [ ]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment.

Summarize the review below, delimited by triple
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The panda plush toy arrived a day earlier than expected, allowing extra time to inspect it before gifting.


### Summarize with focus on the price and value

In [ ]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.

Summarize the review below, delimited by triple
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

The panda plush toy is soft and cute, but perceived as overpriced due to its small size compared to other options.


### Extract information for the shipping department

In [ ]:
prompt = f"""
Your task is to extract relevant information from \
a product review from an ecommerce site to give \
feedback to the Shipping department.

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \
delivery. Limit to 30 words.

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

Arrived a day earlier than expected, allowing time to inspect and play with it before gifting.


### Summarizing many reviews

In [ ]:
review_1 = prod_review

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products.
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean!
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]

In [ ]:
import time
for i in range(len(reviews)):
  prompt = f"""
  Your task is to generate a short summary of a product \
  review from an ecommerce site.

  Summarize the review below, delimited by triple \
  backticks in at most 50 words.

  Review: ```{reviews[i]}```
  """

  response = get_completion(prompt)
  time.sleep(20)
  print(i, response, "\n")

0 The panda plush toy is soft, cute, and has a friendly face, making it a beloved gift for the reviewer's daughter. However, it is smaller than expected for the price. It arrived a day early, allowing the reviewer to enjoy it before gifting. 

1 This lamp offers additional storage at an affordable price and was delivered quickly. Despite a broken string and a missing part, the company's responsive customer service promptly resolved the issues, demonstrating their commitment to customer satisfaction. Assembly was easy, reinforcing the positive experience. 

2 The reviewer appreciates the electric toothbrush's impressive battery life and the clean feeling it provides, akin to a dental visit. However, they find the toothbrush head too small and suggest it could benefit from varied bristle lengths. They recommend purchasing it around $50 and using generic replacement heads for cost efficiency. 

3 The 17-piece system was initially on sale for $49 but increased to $70-$89 in December. The 1

## Inferring

In [ ]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

### Sentiment

In [ ]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive. The reviewer expresses satisfaction with the lamp's price, the company's quick response to issues, and the overall customer service experience.


In [ ]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Give your answer as a single word, either "positive" or "negative"

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


### Identify types of emotions

In [ ]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

satisfaction, gratitude, appreciation, relief, trust


### Identify anger

In [ ]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


## Information extraction

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "Item": "lamp",
  "Brand": "Lumina"
}
```


### Doing multiple tasks at once

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

```json
{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}
```


### Infering topics

In [ ]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that NASA was the most popular
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings,
stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and
incredible opportunities. I'm proud to be a part of
such an innovative organization."

The results were also welcomed by NASA's management team,
with Director Tom Johnson stating, "We are thrilled to
hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly
to achieve our goals, and it's fantastic to see that their
hard work is paying off."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""

In [ ]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long.

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

NASA, Employee Satisfaction, Survey Results, Social Security Administration, Government Response


In [ ]:
# We can split the topics in a python list
response.split(sep=',')

['NASA',
 ' Employee Satisfaction',
 ' Survey Results',
 ' Social Security Administration',
 ' Government Response']

In [ ]:
topic_list = [
    "nasa", "local government", "engineering",
    "employee satisfaction", "federal government"
]

### Classify a text based on topic

#### Make a news alert for certain topics

In [ ]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as follows:
item from the list: 0 or 1

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1  
local government: 0  
engineering: 0  
employee satisfaction: 1  
federal government: 1


In [ ]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


In [ ]:
# Update the prompt above to return dictionary
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as a dictionary with the topic as the \
key and either 0 or 1 as the value.

Topics: ```{topic_list}```

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

```python
{
    'nasa': 1,
    'local government': 0,
    'engineering': 0,
    'employee satisfaction': 1,
    'federal government': 1
}
```


In [ ]:
# Make an alert for each topic
for key,value in topic_dict.items():
  if value == 1:
    print(f"ALERT: New {key} story!")

ALERT: New nasa story!
ALERT: New employee satisfaction story!
ALERT: New federal government story!


## Transformation

### Translation

In [ ]:
prompt = f"""
Translate the following English text to Spanish: \
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)

Hola, me gustaría pedir una licuadora.


### Infer the text language

In [ ]:
prompt = f"""
Tell me which language this is:
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

The sentence "Combien coûte le lampadaire?" is in French. It translates to "How much does the street lamp cost?" in English.


### Translate to many languages

In [ ]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

**French:** Je veux commander un ballon de basket.

**Spanish:** Quiero pedir un balón de baloncesto.

**English Pirate:** Arrr, I be wantin' to order a basketball, matey!


In [ ]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms:
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)

Informal: ¿Te gustaría pedir una almohada?

Formal: ¿Le gustaría pedir una almohada?


### Universal translator

In [ ]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
]

In [ ]:
# Translate each response to English
for issue in user_messages:
  prompt = f"Tell me what language this is: ```{issue}```"
  lang = get_completion(prompt)
  print(f"Original message ({lang}): {issue}")

  prompt = f"""
  Translate the following  text to English \
  and Korean: ```{issue}```
  """
  response = get_completion(prompt)
  print(response, "\n")

Original message (The language of the text is French.): La performance du système est plus lente que d'habitude.
**English:** The system's performance is slower than usual.

**Korean:** 시스템의 성능이 평소보다 느립니다. 

Original message (The language is Spanish.): Mi monitor tiene píxeles que no se iluminan.
**English:** My monitor has pixels that do not light up.

**Korean:** 제 모니터에 불이 들어오지 않는 픽셀이 있습니다. 

Original message (The language is Italian.): Il mio mouse non funziona
**English:** "My mouse is not working."

**Korean:** "내 마우스가 작동하지 않습니다." 

Original message (The sentence "Mój klawisz Ctrl jest zepsuty" is in Polish.): Mój klawisz Ctrl jest zepsuty
**English:** My Ctrl key is broken.

**Korean:** 제 Ctrl 키가 고장 났어요. 

Original message (The text you provided is in Chinese.): 我的屏幕在闪烁
English: "My screen is flickering."

Korean: "내 화면이 깜빡거리고 있어요." 



In [ ]:
# Return the results in a table
# Translate each response to English
for issue in user_messages:
  prompt = f"Tell me what language this is: ```{issue}```"
  lang = get_completion(prompt)
  print(f"Original message ({lang}): {issue}")

  prompt = f"""
  Translate the following  text to English, Korean, and pidgin: ```{issue}```

  Present your result in a table
  """
  response = get_completion(prompt)
  print(response, "\n")

Original message (The language of the text is French.): La performance du système est plus lente que d'habitude.
Here is the translation of the text into English, Korean, and Pidgin presented in a table:

| Language | Translation                                      |
|----------|--------------------------------------------------|
| English  | The system's performance is slower than usual.   |
| Korean   | 시스템의 성능이 평소보다 느립니다.                |
| Pidgin   | Di system dey perform slow pass how e dey be before. | 

Original message (The sentence is in Spanish.): Mi monitor tiene píxeles que no se iluminan.
Sure, here is the translation of the text "Mi monitor tiene píxeles que no se iluminan." into English, Korean, and Pidgin, presented in a table:

| Language | Translation                                  |
|----------|----------------------------------------------|
| English  | My monitor has pixels that do not light up.  |
| Korean   | 내 모니터에는 불이 들어오지 않는 픽셀이 있습니다. |
| Pidgin   | My moni

In [ ]:
# Return the results in a table
# Translate each response to English
for issue in user_messages:
  prompt = f"Tell me what language this is: ```{issue}```"
  lang = get_completion(prompt)
  print(f"Original message ({lang}): {issue}")

  prompt = f"""
  Translate the following  text to English, Korean, and pidgin: ```{issue}```

  Present your result in a table with the following headings `text, language, korean, english, pidgin`
  """
  response = get_completion(prompt)
  print(response, "\n")

Original message (The language of the sentence is French.): La performance du système est plus lente que d'habitude.
Here is the translation of the text into English, Korean, and Pidgin, presented in a table format:

| text                                      | language | korean                          | english                          | pidgin                          |
|-------------------------------------------|----------|---------------------------------|----------------------------------|---------------------------------|
| La performance du système est plus lente que d'habitude. | French   | 시스템의 성능이 평소보다 느립니다. | The system's performance is slower than usual. | Di system dey perform slower than usual. | 

Original message (The language of the text is Spanish.): Mi monitor tiene píxeles que no se iluminan.
Here is the translation of the text into English, Korean, and Pidgin, presented in a table format:

| text                                    | language | korean            

In [ ]:
# Return the results in a table
# Translate each response to English
for issue in user_messages:
  #prompt = f"Tell me what language this is: ```{issue}```"
  lang = get_completion(prompt)
  #print(f"Original message ({lang}): {issue}")

  prompt = f"""
  Translate the following  text to English, Korean, and pidgin: ```{issue}```

  Present your result in an html table with the following headings `text, language, korean, english, pidgin`. Wrap the text if possible.
  """
  response = get_completion(prompt)
  #print(response, "\n")
  from IPython.display import display, Markdown, Latex, HTML, JSON
  display(HTML(response))

text,language,korean,english,pidgin
La performance du système est plus lente que d'habitude.,French,시스템의 성능이 평소보다 느립니다.,The system's performance is slower than usual.,Di system dey slow pass how e dey before.


text,language,korean,english,pidgin
Mi monitor tiene píxeles que no se iluminan.,Spanish,내 모니터에는 켜지지 않는 픽셀이 있습니다.,My monitor has pixels that do not light up.,My monitor get pixels wey no dey light up.


text,language,korean,english,pidgin
Il mio mouse non funziona,Italian,내 마우스가 작동하지 않습니다,My mouse is not working,My mouse no dey work


text,language,korean,english,pidgin
Mój klawisz Ctrl jest zepsuty,Polish,내 Ctrl 키가 고장 났어요,My Ctrl key is broken,My Ctrl key don spoil


text,language,korean,english,pidgin
我的屏幕在闪烁,Chinese,내 화면이 깜빡거려요,My screen is flickering,My screen dey blink


In [ ]:
# Return the results in a table
# Translate each response to English
for issue in user_messages:
  prompt = f"Tell me what language this is: ```{issue}```"
  lang = get_completion(prompt)
  print(f"Original message ({lang}): {issue}")

  prompt = f"""
  Translate the following  text to English, Korean, and pidgin: ```{issue}```

  Present your result in json format with the following keys `text, language, korean, english, pidgin`. Wrap the text if possible
  """
  response = get_completion(prompt)
  print(response, "\n")

Original message (The language of the text is French.): La performance du système est plus lente que d'habitude.
```json
{
  "text": "La performance du système est plus lente que d'habitude.",
  "language": "French",
  "korean": "시스템의 성능이 평소보다 느립니다.",
  "english": "The system's performance is slower than usual.",
  "pidgin": "Di system dey slow pass as e dey before."
}
``` 

Original message (The language of the text is Spanish.): Mi monitor tiene píxeles que no se iluminan.
```json
{
  "text": "Mi monitor tiene píxeles que no se iluminan.",
  "language": "Spanish",
  "korean": "내 모니터에는 켜지지 않는 픽셀이 있습니다.",
  "english": "My monitor has pixels that do not light up.",
  "pidgin": "My monitor get pixels wey no dey shine."
}
``` 

Original message (The language is Italian.): Il mio mouse non funziona
```json
{
  "text": "Il mio mouse non funziona",
  "language": "Italian",
  "korean": "내 마우스가 작동하지 않습니다",
  "english": "My mouse is not working",
  "pidgin": "My mouse no dey work"
}
``` 

Origi

### Tone translation

In [ ]:
prompt = f"""
Translate the following from slang to a business letter:
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Subject: Introduction and Product Specification for Standing Lamp

Dear [Recipient's Name],

I hope this message finds you well. My name is Joe, and I would like to bring to your attention the specifications of a standing lamp that may be of interest to you.

Please find the detailed specifications attached for your review.

Thank you for your time and consideration.

Best regards,

Joe


### Format Conversion

In [ ]:
data_json = { "resturant employees" :[
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

To convert the given JSON-like Python dictionary into an HTML table, you can follow these steps. Below is the HTML code that represents the data in a table format with column headers and a title:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Restaurant Employees</title>
    <style>
        table {
            width: 50%;
            border-collapse: collapse;
            margin: 20px 0;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: #f2f2f2;
        }
    </style>
</head>
<body>

    <h1>Restaurant Employees</h1>
    <table>
        <thead>
            <tr>
                <th>Name</th>
                <th>Email</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>Shyam</td>
                <td>shyamjaiswal@gm

In [ ]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


### Spellcheck / Grammar Check

In [ ]:
# Sentences
text = [
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]

In [ ]:
for t in text:
  prompt = f"""Proofread and correct the following text
  and rewrite the corrected version. If you don't find
  and errors, just say "No errors found". Don't use
  any punctuation around the text:
  ```{t}```"""
  response = get_completion(prompt)
  print(response)

The girl with the black and white puppies has a ball.
No errors found
Its going to be a long day. Does the car need its oil changed?
There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
This phrase is to check ChatGPT for spelling ability


In [ ]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

Here's a proofread and corrected version of your review:

"I got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think it was designed to be asymmetrical. It's a bit small for what I paid for it, though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before giving it to my daughter."


### Get the difference between the original and modified text

In [ ]:
!pip install redlines

In [ ]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>Here's a proofread and corrected version of your review: </span>

<span style='color:green;font-weight:700;'>"I got </span>this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults <span style='color:red;font-weight:700;text-decoration:line-through;'>also </span>like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too. </span>She takes it everywhere with her, and it's super soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  </span><span style='color:green;font-weight:700;'>cute. </span>One of the ears is a bit lower than the other, and I don't think <span style='color:red;font-weight:700;text-decoration:line-through;'>that </span><span style='color:green;font-weight:700;'>it </span>was designed to be asymmetrical. It's a bit small for what I paid for <span style='color:red;font-weight:700;text-decoration:line-through;'>it </span><span style='color:green;font-weight:700;'>it, </span>though. I think there might be other options that are bigger for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  </span><span style='color:green;font-weight:700;'>price. </span>It arrived a day earlier than expected, so I got to play with it myself before <span style='color:red;font-weight:700;text-decoration:line-through;'>I gave </span><span style='color:green;font-weight:700;'>giving </span>it to my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter.</span><span style='color:green;font-weight:700;'>daughter."</span>

In [ ]:
prompt = f"""
proofread and correct this review. Make it more compelling.
Ensure it follows APA style guide and targets an advanced reader.
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

## Expanding
Writing more text from short text

### Writing a custom email to a customer based on their review

In [ ]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [ ]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Subject: Thank You for Your Feedback

Dear [Customer's Name],

Thank you for taking the time to share your detailed review with us. We sincerely apologize for any inconvenience you experienced with the pricing changes and the quality of the product. We understand how frustrating it can be when expectations are not met, especially regarding product durability and pricing.

We appreciate your insights and the special tips you shared for using the system effectively. Your feedback is invaluable to us, and we are committed to improving our products and services.

If you have any further concerns or need assistance, please do not hesitate to reach out to our customer service team. We are here to help and ensure your satisfaction.

Thank you once again for your feedback.

Best regards,

AI Customer Agent


### Remind the model to use details from the customer's email

In [ ]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service.
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

Subject: Thank You for Your Feedback

Dear [Customer's Name],

Thank you for taking the time to share your experience regarding the purchase of our 17-piece and 11-piece systems. We sincerely apologize for any inconvenience caused by the unexpected price changes during the holiday season and the issues you encountered with the product's quality.

We strive to ensure our products meet the highest standards, and it's regrettable to hear that this was not your experience. Your feedback is invaluable and will help us improve our offerings. 

If you have any further concerns or need assistance, please feel free to reach out to our customer service team. We are committed to addressing any issues you may have.

Thank you once again for your feedback.

Best regards,

AI Customer Agent


## Creating a chatbot

In [ ]:
def get_completion(prompt, model="gpt-4o", temperature=0):
  response = client.chat.completions.create(
    model=model,
    messages = [{"role": "user", "content": prompt}],
    temperature=temperature, # this is the degree of randomness of the model's output. The higher the more random
    store=True,
  )
  return response.choices[0].message.content

In [ ]:
def get_completion_from_messages(messages, model="gpt-4o", temperature=0):
  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature, # this is the degree of randomness of the model's output
  )
  #print(str(response.choices[0].message))
  return response.choices[0].message.content

In [ ]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To reach yon fair side whence lies the fowl's abode!


In [ ]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello, Isa! It's nice to meet you. How can I assist you today?


In [ ]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I can't recall personal information like your name. If you tell me, I'll be happy to address you by it during our conversation!


In [ ]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Of course, your name is Isa! If there's anything else you need, feel free to ask.


## Creating an OrderBot

In [ ]:
!pip install inp
!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
def collect_messages(_):
  prompt = inp.value_input
  inp.value = ''
  context.append({'role':'user', 'content':f"{prompt}"})
  response = get_completion_from_messages(context)
  context.append({'role':'assistant', 'content':f"{response}"})
  panels.append(
  pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
  panels.append(
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

  return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

In [ ]:
panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


In [ ]:
inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, defer_load=True,height=300),
)

dashboard.

TypeError: Object of type TextInput is not JSON serializable

Tabs
    [0] WidgetBox(margin=(5, 10), name='Controls')
        [0] StaticText(value='<b>Controls</b>')
        [1] ListInput(description='The list of c..., name='Objects', serializer='json', type=<class 'list'>, value=[TextInput(placeholder='En...])
        [2] IntInput(description='Max pixel distance f..., name='Auto scroll limit', start=0)
        [3] IntInput(description='Min pixel distance f..., name='Scroll button threshold', start=0)
        [4] IntInput(description='Current scroll p..., name='Scroll position')
        [5] Checkbox(name='View latest')
    [1] WidgetBox(margin=(5, 10), name='Layout')
        [0] StaticText(value='<b>Layout</b>')
        [1] TextInput(description='String identifier f..., name='Name', value='Column00263')
        [2] LiteralInput(description='Whether the object should..., name='Align', serializer='json', value='start')
        [3] LiteralInput(description='Describes the proportiona..., name='Aspect ratio', serializer='json')
        [4] ListInput(d

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

I'm sorry, but it seems there hasn't been an order placed yet. Could you please let me know what you'd like to order, and I'll be happy to create a summary for you!
